# Description

TODO

# Environment variables

In [1]:
from IPython.display import display

import conf

N_JOBS = conf.GENERAL["N_JOBS"]
display(N_JOBS)

3

In [2]:
%env MKL_NUM_THREADS=$N_JOBS
%env OPEN_BLAS_NUM_THREADS=$N_JOBS
%env NUMEXPR_NUM_THREADS=$N_JOBS
%env OMP_NUM_THREADS=$N_JOBS

env: MKL_NUM_THREADS=3
env: OPEN_BLAS_NUM_THREADS=3
env: NUMEXPR_NUM_THREADS=3
env: OMP_NUM_THREADS=3


# Modules loading

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from pathlib import Path

import numpy as np
import pandas as pd

# from sklearn.cluster import SpectralClustering
# import umap
import matplotlib.pyplot as plt
import seaborn as sns

from utils import generate_result_set_name

# Settings

In [5]:
RANDOM_GENERATOR = np.random.default_rng(12345)

## Ensemble size

In [6]:
EXPECTED_ENSEMBLE_SIZE = 295

MIN_ENSEMBLE_SIZE = 290
MAX_ENSEMBLE_SIZE = 300

## Data subsets included

In [7]:
# DATA_SUBSETS = [
#     'z_score_std',
#     'pca',
#     'umap'
# ]

## Algorithms included

In [8]:
# ALGORITHMS = [
#     'KMeans',
#     COMPLETE
# ]

## Input data

In [9]:
# DATA_SUBSET = 'z_score_std'

# INPUT_DIR = Path(
#     conf.RESULTS["DATA_TRANSFORMATIONS_DIR"],
#     DATA_SUBSET,
# ).resolve()
# display(INPUT_DIR)

In [10]:
# INPUT_STEM = 'projection-smultixcan-efo_partial-mashr-zscores'

In [11]:
# INPUT_FILEPATH = Path(
#     INPUT_DIR,
#     generate_result_set_name(
#         {},
#         prefix=f'{DATA_SUBSET}-{INPUT_STEM}',
#         suffix='.pkl'
#     )
# ).resolve()
# display(INPUT_FILEPATH)

# assert INPUT_FILEPATH.exists(), 'Input file does not exist'

## Consensus clustering

In [12]:
# ALL_RUNS_OPTIONS = {}

# ALL_RUNS_OPTIONS['K_MIN'] = 2
# ALL_RUNS_OPTIONS['K_MAX'] = 25
# ALL_RUNS_OPTIONS['N_REPS_PER_K'] = 100
# ALL_RUNS_OPTIONS['KMEANS_N_INIT'] = 1
# ALL_RUNS_OPTIONS['AFFINITY'] = 'nearest_neighbors'
# ALL_RUNS_OPTIONS['N_JOBS'] = N_JOBS

# display(ALL_RUNS_OPTIONS)

In [13]:
# output dir for this notebook
RESULTS_DIR = Path(conf.RESULTS["CLUSTERING_DIR"], "consensus_clustering").resolve()
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

display(RESULTS_DIR)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base3/results/clustering/consensus_clustering')

# Get ensemble

## Load partition files

In [14]:
input_dir = Path(
    conf.RESULTS["CLUSTERING_RUNS_DIR"],
).resolve()
display(input_dir)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base3/results/clustering/runs')

In [15]:
included_pkl_files = []

for pkl_file in input_dir.rglob("*.pkl"):
    pkl_file_str = str(pkl_file)

    if "-stability-" in pkl_file_str:
        continue

    included_pkl_files.append(pkl_file)

In [16]:
display(len(included_pkl_files))
# 5 algorithms, 3 dataset versions
assert len(included_pkl_files) == 5 * 3

15

## Combine partition files to get final ensemble

In [17]:
n_partitions = 0

In [18]:
ensembles_list = []

In [19]:
for ens_file in included_pkl_files:
    ens = pd.read_pickle(ens_file)
    #     ens_by_k = ens.groupby('n_clusters').apply(
    #         lambda x: np.concatenate(x['partition'].apply(lambda x: x.reshape(1, -1)), axis=0)
    #     )
    short_file_path = Path(*ens_file.parts[-2:])

    if ens.shape[0] < MIN_ENSEMBLE_SIZE:
        print(f"Less partitions than expected in {short_file_path}")

        ens = ens.sample(
            n=EXPECTED_ENSEMBLE_SIZE,
            replace=True,
            random_state=RANDOM_GENERATOR.bit_generator,
        )
        assert ens.shape[0] == EXPECTED_ENSEMBLE_SIZE

    elif ens.shape[0] > MAX_ENSEMBLE_SIZE:
        print(f"More partitions than expected in {short_file_path}")

        ens = ens.sample(
            n=EXPECTED_ENSEMBLE_SIZE, random_state=RANDOM_GENERATOR.bit_generator
        )
        assert ens.shape[0] == EXPECTED_ENSEMBLE_SIZE

    ens_full_format = np.concatenate(
        ens["partition"].apply(lambda x: x.reshape(1, -1)), axis=0
    )

    # check ensemble size
    #     n_parts = ensemble_full_format.shape[0]
    #     if n_parts > MAX_ENSEMBLE_SIZE:

    n_partitions += ens_full_format.shape[0]

    ensembles_list.append(ens_full_format)

More partitions than expected in umap-z_score_std-projection-smultixcan-efo_partial-mashr-zscores/DBSCAN.pkl
Less partitions than expected in z_score_std-projection-smultixcan-efo_partial-mashr-zscores/DBSCAN.pkl
Less partitions than expected in pca-z_score_std-projection-smultixcan-efo_partial-mashr-zscores/DBSCAN.pkl


In [20]:
display(len(ensembles_list))
assert len(ensembles_list) == len(included_pkl_files)

15

In [21]:
n_data_objects = ensembles_list[0].shape[1]
display(n_data_objects)

3749

In [22]:
display(n_partitions)

4428

In [23]:
full_ensemble = ensembles_list[0]
for ens in ensembles_list[1:]:
    full_ensemble = np.concatenate((full_ensemble, ens), axis=0)

In [24]:
display(full_ensemble.shape)
assert full_ensemble.shape == (n_partitions, n_data_objects)

(4428, 3749)

## Save

In [25]:
output_file = Path(RESULTS_DIR, "ensemble.npy").resolve()
display(output_file)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base3/results/clustering/consensus_clustering/ensemble.npy')

In [26]:
full_ensemble

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [27]:
np.save(output_file, full_ensemble)

# Get coassociation matrix from ensemble

In [28]:
from clustering.ensemble import get_ensemble_distance_matrix

In [29]:
ensemble_coassoc_matrix = get_ensemble_distance_matrix(
    full_ensemble,
    n_jobs=conf.GENERAL["N_JOBS"],
)

In [30]:
ensemble_coassoc_matrix.shape

(3749, 3749)

In [31]:
ensemble_coassoc_matrix

array([[0.        , 0.05004529, 0.04868659, ..., 0.62794384, 0.66445714,
        0.67006341],
       [0.05004529, 0.        , 0.02241848, ..., 0.62771739, 0.66514286,
        0.66779891],
       [0.04868659, 0.02241848, 0.        , ..., 0.62703804, 0.66285714,
        0.66530797],
       ...,
       [0.62794384, 0.62771739, 0.62703804, ..., 0.        , 0.59288808,
        0.66282746],
       [0.66445714, 0.66514286, 0.66285714, ..., 0.59288808, 0.        ,
        0.6127194 ],
       [0.67006341, 0.66779891, 0.66530797, ..., 0.66282746, 0.6127194 ,
        0.        ]])

## Save

In [32]:
output_file = Path(RESULTS_DIR, "ensemble_coassoc_matrix.npy").resolve()
display(output_file)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base3/results/clustering/consensus_clustering/ensemble_coassoc_matrix.npy')

In [33]:
np.save(output_file, ensemble_coassoc_matrix)